### Preamble

In [76]:
# @title Preamble
import numpy as np
import math
import plotly.graph_objects as go
import matplotlib.pyplot as plt

### Functions

In [77]:
def run_kut4(F,x,y,h):
    K1 = F(x,y)
    K2 = F(x + h/2.0, y + K1/2.0)
    K3 = F(x + h/2.0, y + K2/2.0)
    K4 = F(x + h, y + K3)
    return (h/6.0)*(K1 + 2.0*K2 + 2.0*K3 + K4)

def integrate(F,x,y,xStop,h):
    X = []
    Y = []
    X.append(x)
    Y.append(y)
    while x < xStop:
        h = min(h,xStop - x)
        y = y + run_kut4(F,x,y,h)
        x=x+h
        X.append(x)
        Y.append(y)
    return np.array(X),np.array(Y)

In [78]:
def runge_kutta_4(f, x0, y0, z0, t0, t_end, dt):
    # Number of steps
    n = math.ceil((t_end - t0) / dt)
    
    # Arrays to store the solutions at each time step
    t = np.zeros(n+1)
    x = np.zeros(n+1)
    y = np.zeros(n+1)
    z = np.zeros(n+1)
    
    # Initial conditions
    x[0] = x0
    y[0] = y0
    z[0] = z0
    t[0] = t0
    
    # Runge-Kutta 4th order integration
    for i in range(n):  
        k1T, k1B, k1M = f(
            x[i], 
            y[i], 
            z[i], 
        )
        k2T, k2B, k2M = f(
            x[i] + 0.5*dt*k1T, 
            y[i] + 0.5*dt*k1B, 
            z[i] + 0.5*dt*k1M
        )
        k3T, k3B, k3M = f(
            x[i] + 0.5*dt*k2T, 
            y[i] + 0.5*dt*k2B, 
            z[i] + 0.5*dt*k2M
        )
        k4T, k4B, k4M = f(
            x[i] + dt*k3T, 
            y[i] + dt*k3B, 
            z[i] + dt*k3M
        )
        
        x[i+1] = x[i] + (dt/6)*(k1T + 2*k2T + 2*k3T + k4T)
        y[i+1] = y[i] + (dt/6)*(k1B + 2*k2B + 2*k3B + k4B)
        z[i+1] = z[i] + (dt/6)*(k1M + 2*k2M + 2*k3M + k4M)
        t[i+1] = t[i] + dt
        
    return t, x, y, z

### Evaluation

In [79]:
def system(T, B, M):
    # constants 
    weight = 70
    bodyfat = 0.20

    # list of functions 
    dTdt = -10*(1/weight)*T + 0.05*bodyfat*B
    dBdt = 3.8*(1/weight)*T - 0.11*bodyfat*B
    dMdt = ((1/weight) - bodyfat*(1/weight)) * 3.8 * T - (bodyfat-(bodyfat ** 2))* 0.11 * B

    return dTdt, dBdt, dMdt

In [80]:
# Initial conditions
dosage, y0, z0 = 5.0, 0.0, 0.0
t0 = 0.0
t_end = 100.0
dt = 0.01

# Solve the system using the Runge-Kutta method
t, x, y, z = runge_kutta_4(system, dosage, y0, z0, t0, t_end, dt)

### Plotting

In [81]:
# Plot the results
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=t, y=x, mode='lines', marker = {'color' : 'red'}, name="Trapezius"),
    go.Scatter(x=t, y=y, mode='lines', marker = {'color' : 'blue'}, name="Bloodstream"),
    go.Scatter(x=t, y=z, mode='lines', marker = {'color' : 'green'}, name="Other muscles")
])
fig.update_layout(
    title_text='Clenbuterol presence in body over time',
    xaxis_title='Time (hours)',
    yaxis_title='Concentration (mcg/L)',
    height=1080*0.5,
    width=1920*0.6,
    font_family="CMU Serif",
    font_size=15,
    title_font_size=25,
    font_color="#0e0f11",
    margin=dict(t=120, b=80)
)
fig.show()

In [82]:
# plt.figure(figsize=(12, 8))
# plt.plot(t, x, label='T(t)')
# plt.plot(t, y, label='B(t)')
# plt.plot(t, z, label='M(t)')
# plt.xlabel('Time t')
# plt.ylabel('Solutions x(t), y(t), z(t)')
# plt.legend()
# plt.title('Solutions of the system of differential equations using 4th order Runge-Kutta method')
# plt.show()